In [4]:
import h5py


file_path = "./TimeSeries_Scenario_00003_ScenarioInstanceRandint_id_0003.h5"
with h5py.File(file_path, 'r') as hdf:
    # Liệt kê các nhóm dữ liệu trong file
    for group in hdf.keys():
        print(f"Group: {group}")
        for dataset in hdf[group].keys():
            print(f"  Dataset: {dataset}")
            # Đọc dữ liệu trong dataset
            data = hdf[group][dataset][:]
            print(data)


Group: data
  Dataset: axis0
[b'sim/time/total_running_time_sec' b'sim/time/total_flight_time_sec'
 b'sim/aircraft/overflow/acf_nosewheel_k' ...
 b'sim/cockpit2/tcas/targets/position/weight_on_wheels61'
 b'sim/cockpit2/tcas/targets/position/weight_on_wheels62'
 b'sim/cockpit2/tcas/targets/position/weight_on_wheels63']
  Dataset: axis1
[   0    1    2 ... 3579 3580 3581]
  Dataset: block0_items
[b'sim/time/total_running_time_sec' b'sim/time/total_flight_time_sec'
 b'sim/aircraft/overflow/acf_nosewheel_k' ...
 b'sim/cockpit2/hydraulics/indicators/hydraulic_fluid_ratio_2'
 b'sim/cockpit2/hydraulics/indicators/hydraulic_pressure_1'
 b'sim/cockpit2/hydraulics/indicators/hydraulic_pressure_2']
  Dataset: block0_values
[[1.0809464e+03 2.5125629e-01 0.0000000e+00 ... 8.5000002e-01
  2.7494722e+03 2.7494722e+03]
 [1.0810470e+03 3.5175881e-01 0.0000000e+00 ... 8.5000002e-01
  2.7155488e+03 2.7155488e+03]
 [1.0811475e+03 4.5226130e-01 0.0000000e+00 ... 8.5000002e-01
  2.7048867e+03 2.7048867e+03]